Importing essential packages

In [4]:
# importing essential packages
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Loading the dataset into a pandas dataframe

In [5]:

ds = pd.read_csv("train.csv")
print(ds.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


checking the missing values in our dataset and replacing them with an empty string

In [6]:

print(ds.isnull().sum())
ds = ds.fillna('')
print(ds.head())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


 Merging the author and title column as content

In [7]:

ds['content'] = ds['author'] + ' ' + ds['title']
print(ds['content'].head())

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object


Performing stemming on the content

In [8]:

port_stem = PorterStemmer()

In [9]:
def stemming(content):
    stemmed_cont = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_cont = stemmed_cont.lower()
    stemmed_cont = stemmed_cont.split()
    stemmed_cont = [port_stem.stem(word) for word in stemmed_cont if word not in stopwords.words('english')]
    stemmed_cont = " ".join(stemmed_cont)
    return stemmed_cont

In [10]:
ds['content'] = ds['content'].apply(stemming)
print(ds['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


storing the data in x and y from training the model. we will be using content and labels to train our model

In [17]:

x = ds['content']
y = ds['label']
print(x)
print(y)

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Performing feature vectorization to convert textual data to numerical data

In [18]:

vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

creating the training and test data

In [19]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

Implementing the Logistic Regression model

In [20]:

model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

Finding the accuracy score of our model on training data

In [21]:

x_train_predict = model.predict(x_train)
accuracy = accuracy_score(x_train_predict, y_train)
print(accuracy, "is the accuracy score of the training data")

0.9865985576923076 is the accuracy score of the training data


Finding the accuracy score of our model on test data and making prediction on test data sing our model

In [22]:

x_test_predict = model.predict(x_test)
accuracy = accuracy_score(x_test_predict, y_test)
print(accuracy, "is the accuracy score of the test data")

0.9790865384615385 is the accuracy score of the test data


Predicting a news

In [26]:

news = x_test[-1]
prediction = model.predict(news)

if prediction == 1:
    print("This is a Fake news")
else:
    print("This is a Real news")


This is a Real news
